In [1]:
#%%
print('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/nn_model_script_emb_test.py - starting')

from array import array
from cmath import nan
from pyexpat import model
import statistics
from tkinter.ttk import Separator
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms
from torch.autograd import variable
from itertools import chain
from sklearn import metrics as met
import pickle
from icecream import ic

import matplotlib.pyplot as plt
import pathlib
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
# import util
# import model_torch_simple
# from torchmetrics import Accuracy
from tqdm import tqdm
import argparse
from icecream import ic
import numpy as np
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#%%
seed = 42
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)

# train_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/aa_data_train_gene.csv', delimiter = ',')
# train_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/mic_aa_train_hml.csv')
# train_target = train_target[['EMB_MIC']]
# # don't touch test data, split out validation data from training data during training
# # test_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_EMB/aa_data_test_pca4k.csv', delimiter = ',')
# test_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/aa_data_test_gene.csv', delimiter = ',')
# test_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/mic_aa_test_hml.csv')
# test_target = test_target[['EMB_MIC']]

# all_data = np.concatenate((train_data, test_data), axis=0)
# all_target = pd.concat((train_target, test_target), axis=0)

# train_data, test_data, train_target, test_target = train_test_split(all_data, all_target, test_size=0.2, random_state=42, stratify=all_target)

/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/nn_model_script_emb_test.py - starting


In [2]:

def data_split(aa_array, encoded_mic):
    np.random.seed(seed)

    # Perform stratified train-test split
    train_data, test_data, train_target, test_target = train_test_split(
        aa_array,
        encoded_mic,
        test_size=0.1,  # 10% for testing
        stratify=encoded_mic,  # Ensures the proportion of each class is preserved
        random_state=42  # For reproducibility
    )
    return train_data, test_data, train_target, test_target

def is_within_doubling_dilution(pred, target, target_min, target_max):
    _ = np.arange(target_min-1, target_max+2, 1)
    index = [i for i, x in enumerate(_) if x == target][0]
    return (_[index-1] <= pred <= _[index+1])

## 50k

In [3]:
import numpy as np

# Read the CSV file into a NumPy array
aa_array = np.loadtxt('aa_array_50k.csv', delimiter=',').astype(float)  # Ensure all elements are floats
print(aa_array)

# Read the text file line by line into a list and convert to floats
with open('dr_target_50k.txt', 'r') as f:
    mic_series = [float(line.strip()) for line in f]  # Convert each line to float

print(mic_series)
mic_series_50k = mic_series

train_data, test_data, train_target, test_target = data_split(aa_array, mic_series)
# train_target = train_target.to_frame()
# test_target = test_target.to_frame()
train_target = pd.DataFrame(train_target, columns=['a'])
test_target = pd.DataFrame(test_target, columns=['a'])

[[1. 1. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [5]:
aa_array_50k_neg = []
mic_series_50k_neg = []
for x, a in zip(mic_series_50k, aa_array):
    if x ==  0:
        aa_array_50k_neg.append(a)
        mic_series_50k_neg.append(x)
aa_array_50k_neg = np.array(aa_array_50k_neg)

In [6]:
aa_array_50k_pos = []
mic_series_50k_pos = []
for x, a in zip(mic_series_50k, aa_array):
    if x == 1:
        aa_array_50k_pos.append(a)
        mic_series_50k_pos.append(x)
aa_array_50k_pos = np.array(aa_array_50k_pos)

### xgb

In [71]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score

# train_data = np.column_stack((train_data, train_target['EMB_MIC_y'].values))
# test_data = np.column_stack((test_data, test_target['EMB_MIC_y'].values))
train_target_y = train_target['a'].values
test_target_y = test_target['a'].values

def is_within_doubling_dilution(pred, target, target_min, target_max):
    _ = np.arange(target_min-1, target_max+2, 1)
    index = [i for i, x in enumerate(_) if x == target][0]
    return (_[index-1] <= pred <= _[index+1])

# # Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_data, test_data, train_target_y, test_target_y

# Create the XGBoost model
model_bi = xgb.XGBClassifier()
model_bi.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model_bi.predict(X_test)

# Evaluate the model_bi
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

#testing
cutoff = 4
test_target_bi = y_test.astype(int) #(target_mic_list  >= cutoff).astype(int)
test_predictions_bi = y_pred.astype(int)  #(np.array(pred_mic_list) >= cutoff).astype(int)

auc = roc_auc_score(test_target_bi, test_predictions_bi)
print("AUC:", auc)

# Calculate confusion matrix components
tn, fp, fn, tp = confusion_matrix(test_target_bi, test_predictions_bi).ravel()
print(confusion_matrix(test_target_bi, test_predictions_bi))
# Calculate sensitivity (recall)
sensitivity = tp / (tp + fn)
print("Sensitivity:", sensitivity)

# Calculate specificity
specificity = tn / (tn + fp)
print("Specificity:", specificity)

Accuracy: 0.9084018706660216
AUC: 0.5648495635210325
[[11101   141]
 [  995   165]]
Sensitivity: 0.14224137931034483
Specificity: 0.9874577477317203


In [72]:
pd.DataFrame(y_train).value_counts()

0.00    101178
1.00     10436
Name: count, dtype: int64

### mlp

### lazy

In [73]:

# Import necessary libraries
import numpy as np
import pandas as pd
from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Prepare your data
# Assuming train_data, test_data, train_target_y, and test_target_y are already defined

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_data, test_data, train_target_y, test_target_y

# ================================
# Step 1: Use LazyClassifier to Evaluate Models
# ================================

# Initialize LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

# Fit and evaluate models on the train and test data
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Display the evaluation results for each model
print("Model Performance Summary:")
print(models)


 28%|██▊       | 8/29 [27:21<1:11:48, 205.15s/it]


KeyboardInterrupt: 

### selected models

## cryptic

In [88]:
def data_prep_(cryptic, gene_list):
    # overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    # variants = variants[variants['sample_id'].isin(overlap)]
    # variants['SNP'] = variants['gene'] + '-'+ variants['change']
    
    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']

    overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    variants = variants[variants['gene'].isin(gene_list)]
    variants = variants[variants['sample_id'].isin(overlap)]
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    # print(variants.shape)
    # print(variants['sample_id'].unique().shape)

    def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
        # Create a dictionary from query_list and values_list for direct mapping
        query_dict = dict(zip(query_list, values_list))
        
        # Use list comprehension to build the output list directly
        output_list = [query_dict.get(snp, 0) for snp in set_list]
        
        return output_list

    # Example usage
    # set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
    # query_list = ['SNP2', 'SNP4']
    # values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
    # output_list = compare_snp_lists_with_values_optimized(set_list, query_list, values_list)
    # print(output_list)  # Expected output: [0, 5, 0, 10]# Getting all snp data

    aa = []
    all_snp = variants['SNP'].unique() # here is a list of all snps values title for the row in the final table 
    for x in tqdm(overlap):
    # for x in tqdm(variants['sample_id'].unique()):
        if x in variants['sample_id'].tolist():
            aa.append(compare_snp_lists_with_values_optimized(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))
        else:
            aa.append([0]*len(all_snp))
            
        # print('SNP')
        
    aa_array = np.array(aa)
    aa_array[aa_array < 0.8] = 0
    aa_array[aa_array >= 0.8] = 1

    mic_aa = cryptic[cryptic['ENA_RUN'].isin(overlap)]#.iloc[:,14:27]
    # mic_aa = cryptic[cryptic['ENA_RUN'].isin(variants['sample_id'].unique())]#.iloc[:,14:27]
    # print(mic_aa.shape)
    # mic_aa['wgs_id'] = pd.Categorical(mic_aa['ENA_RUN'], categories=variants['sample_id'].unique().tolist(), ordered=True)
    # mic_aa = mic_aa.sort_values('ENA_RUN')
    mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
    mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)
    # mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(variants['sample_id'].unique().tolist())
    mic_aa = mic_aa.sort_values(["ENA_RUN"])  ## 'sort' changed to 'sort_values'
    # print(mic_aa.shape)

    return aa_array, mic_aa

In [89]:
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
gene_list = ['embB', 'embA', 'embC']
df_emb = df[df['EMB_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = 'EMB_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.25':
        df_emb.loc[i, f'{x}'] = '0.125'
        
df_emb['EMB_MIC'] = df_emb['EMB_MIC'].astype('float') 
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array, mic_aa = data_prep_(cryptic, gene_list)
# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

encoded_mic = mic_aa['EMB_MIC'].to_list()

# train_data, test_data, train_target, test_target = data_split(aa_array, encoded_mic)

mic_series = np.log2(mic_aa['EMB_MIC'])
mic_series += 1
sample_ids = mic_aa['ENA_RUN']

mic_series_bi = mic_aa['EMB_MIC'].apply(lambda x: 1 if x >= 8 else 0)
mic_series_sir = mic_aa['EMB_MIC'].apply(lambda x: 2 if x >= 8 else (0 if x <= 2 else 1))
mic_series_all = pd.merge(mic_series, mic_series_bi,left_index=True, right_index=True)
mic_series_all = pd.merge(mic_series_all, mic_series_sir,left_index=True, right_index=True)
train_data, test_data, train_target, test_target = data_split(aa_array, mic_series_all)
target_min, target_max = mic_series.min(), mic_series.max()

/tmp/ipykernel_24123/3993053628.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb.loc[i, f'{x}'] = '16'
/tmp/ipykernel_24123/3993053628.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb['EMB_MIC'] = df_emb['EMB_MIC'].astype('float')
100%|██████████| 11362/11362 [01:02<00:00, 182.12it/s]
/tmp/ipykernel_24123/619452980.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

### xgb

In [90]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score

# train_data = np.column_stack((train_data, train_target['EMB_MIC_y'].values))
# test_data = np.column_stack((test_data, test_target['EMB_MIC_y'].values))
train_target_y = train_target['EMB_MIC_y'].values
test_target_y = test_target['EMB_MIC_y'].values

def is_within_doubling_dilution(pred, target, target_min, target_max):
    _ = np.arange(target_min-1, target_max+2, 1)
    index = [i for i, x in enumerate(_) if x == target][0]
    return (_[index-1] <= pred <= _[index+1])

# # Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_data, test_data, train_target_y, test_target_y

# Create the XGBoost model
model_bi = xgb.XGBClassifier()
model_bi.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model_bi.predict(X_test)

# Evaluate the model_bi
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

#testing
cutoff = 4
test_target_bi = y_test.astype(int) #(target_mic_list  >= cutoff).astype(int)
test_predictions_bi = y_pred.astype(int)  #(np.array(pred_mic_list) >= cutoff).astype(int)

auc = roc_auc_score(test_target_bi, test_predictions_bi)
print("AUC:", auc)

# Calculate confusion matrix components
tn, fp, fn, tp = confusion_matrix(test_target_bi, test_predictions_bi).ravel()
print(confusion_matrix(test_target_bi, test_predictions_bi))

# Calculate sensitivity (recall)
sensitivity = tp / (tp + fn)
print("Sensitivity:", sensitivity)

# Calculate specificity
specificity = tn / (tn + fp)
print("Specificity:", specificity)

Accuracy: 0.8988566402814424
AUC: 0.8226964979207477
[[892  62]
 [ 53 130]]
Sensitivity: 0.7103825136612022
Specificity: 0.9350104821802935


In [ ]:
# import pickle
# file_name = "xgb_bi.pkl"

# # save
# pickle.dump(model_bi, open(file_name, "wb"))

# load
# xgb_model_loaded = pickle.load(open(file_name, "rb"))

### svc - one of the top performing models according to lazy predict    


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report

# Assuming train_data, test_data, train_target_y, and test_target_y are already defined

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_data, test_data, train_target_y, test_target_y

# Initialize the LinearSVC model with calibrated probabilities
base_svc = LinearSVC(max_iter=10000)
model = CalibratedClassifierCV(base_estimator=base_svc, cv=5)  # Calibrate for probability predictions

# Train the model on the training data
print("Training the LinearSVC model with probability calibration...")
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Calculate performance metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

# Confusion matrix and derived metrics
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

# Print the results
print("\nLinearSVC Model Performance:")
print("----------------------------")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC: {auc:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

# Display confusion matrix and classification report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Training the LinearSVC model with probability calibration...

LinearSVC Model Performance:
----------------------------
Accuracy: 0.9129
F1 Score: 0.8511
AUC: 0.9305
Sensitivity (Recall): 0.8524
Specificity: 0.9379

Confusion Matrix:
[[755  50]
 [ 49 283]]

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       805
           1       0.85      0.85      0.85       332

    accuracy                           0.91      1137
   macro avg       0.89      0.90      0.89      1137
weighted avg       0.91      0.91      0.91      1137



### lazy

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Prepare your data
# Assuming train_data, test_data, train_target_y, and test_target_y are already defined

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_data, test_data, train_target_y, test_target_y

# ================================
# Step 1: Use LazyClassifier to Evaluate Models
# ================================

# Initialize LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

# Fit and evaluate models on the train and test data
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Display the evaluation results for each model
print("Model Performance Summary:")
print(models)


 97%|█████████▋| 28/29 [06:08<00:08,  8.79s/it]

[LightGBM] [Info] Number of positive: 2988, number of negative: 7237
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023802 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 144
[LightGBM] [Info] Number of data points in the train set: 10225, number of used features: 48
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.292225 -> initscore=-0.884602
[LightGBM] [Info] Start training from score -0.884602


100%|██████████| 29/29 [06:10<00:00, 12.78s/it]

Model Performance Summary:
                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score   
Model                                                                           
LinearSVC                          0.91               0.90     0.90      0.91  \
AdaBoostClassifier                 0.91               0.90     0.90      0.91   
CalibratedClassifierCV             0.91               0.90     0.90      0.91   
RidgeClassifierCV                  0.91               0.90     0.90      0.91   
RidgeClassifier                    0.91               0.90     0.90      0.91   
LinearDiscriminantAnalysis         0.91               0.90     0.90      0.91   
RandomForestClassifier             0.91               0.89     0.89      0.91   
ExtraTreesClassifier               0.91               0.89     0.89      0.91   
DecisionTreeClassifier             0.91               0.89     0.89      0.91   
ExtraTreeClassifier                0.91               0.89     0.89      0.91   
L

### selected models

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import RidgeClassifierCV, RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report

# Assuming train_data, test_data, train_target_y, and test_target_y are already defined

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_data, test_data, train_target_y, test_target_y

# Define the models to evaluate
models = {
    'LinearSVC': LinearSVC(max_iter=10000),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'CalibratedClassifierCV': CalibratedClassifierCV(base_estimator=LinearSVC(max_iter=10000), cv=5),
    'RidgeClassifierCV': RidgeClassifierCV(),
    'RidgeClassifier': RidgeClassifier(),
    'LinearDiscriminantAnalysis': LinearDiscriminantAnalysis()
}

# Initialize an empty list to store metric dictionaries for each model
results_list = []

# Iterate over models and evaluate each
for model_name, model in models.items():
    print(f"\nTraining and evaluating {model_name}...")
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # If the model has a predict_proba method, use it for AUC calculation
    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    else:
        # Use decision function for AUC calculation if predict_proba is not available
        y_pred_proba = model.decision_function(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    # Confusion matrix and derived metrics
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    # Store results in a dictionary
    results_list.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'F1 Score': f1,
        'AUC': auc,
        'Sensitivity': sensitivity,
        'Specificity': specificity
    })
    
    # Display confusion matrix
    print(f"Confusion Matrix for {model_name}:\n", confusion_matrix(y_test, y_pred))
    print(f"Classification Report for {model_name}:\n", classification_report(y_test, y_pred))

# Convert results list to DataFrame
results = pd.DataFrame(results_list)

# Display overall results
print("\nSummary of Model Performance:")
print(results)



Training and evaluating LinearSVC...
Confusion Matrix for LinearSVC:
 [[755  50]
 [ 48 284]]
Classification Report for LinearSVC:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94       805
           1       0.85      0.86      0.85       332

    accuracy                           0.91      1137
   macro avg       0.90      0.90      0.90      1137
weighted avg       0.91      0.91      0.91      1137


Training and evaluating AdaBoostClassifier...
Confusion Matrix for AdaBoostClassifier:
 [[751  54]
 [ 47 285]]
Classification Report for AdaBoostClassifier:
               precision    recall  f1-score   support

           0       0.94      0.93      0.94       805
           1       0.84      0.86      0.85       332

    accuracy                           0.91      1137
   macro avg       0.89      0.90      0.89      1137
weighted avg       0.91      0.91      0.91      1137


Training and evaluating CalibratedClassifierCV...
Conf

### ensesmble   - bad idea - lowers sensitivity and specificity


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier, VotingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import RidgeClassifierCV, RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report

# Initialize models with appropriate configuration
models = [
    ('LinearSVC', CalibratedClassifierCV(base_estimator=LinearSVC(max_iter=10000), cv=5)),
    ('AdaBoost', AdaBoostClassifier()),
    ('RidgeClassifierCV', RidgeClassifierCV()),
    ('RidgeClassifier', RidgeClassifier()),
    ('LDA', LinearDiscriminantAnalysis())
]

# Create a VotingClassifier ensemble with hard voting
ensemble = VotingClassifier(estimators=models, voting='hard')

# Train the ensemble
print("Training the ensemble model with hard voting...")
ensemble.fit(X_train, y_train)

# Make predictions on the test set
y_pred = ensemble.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
# Hard voting does not provide probabilities, so AUC cannot be calculated
auc = "N/A"

# Confusion matrix and derived metrics
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

# Print the results
print("\nEnsemble Model Performance (Hard Voting):")
print("---------------------------")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC: {auc}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

# Display confusion matrix and classification report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Training the ensemble model with hard voting...

Ensemble Model Performance (Hard Voting):
---------------------------
Accuracy: 0.9129
F1 Score: 0.8511
AUC: N/A
Sensitivity (Recall): 0.8524
Specificity: 0.9379

Confusion Matrix:
[[755  50]
 [ 49 283]]

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       805
           1       0.85      0.85      0.85       332

    accuracy                           0.91      1137
   macro avg       0.89      0.90      0.89      1137
weighted avg       0.91      0.91      0.91      1137



## grid search for xgb parameters

In [111]:
# Import necessary libraries
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report


# ================================

param_grid = {
    'n_estimators': [50, 100, 200],            # Number of trees in the ensemble
    'max_depth': [3, 5, 7],                    # Maximum tree depth
    'learning_rate': [0.01, 0.05, 0.1],        # Learning rate (eta)
    'subsample': [0.6, 0.8, 1.0],              # Subsample ratio of the training instances
    'colsample_bytree': [0.6, 0.8, 1.0],       # Subsample ratio of columns when constructing each tree
    'gamma': [0, 0.1, 0.2],                    # Minimum loss reduction required for a split
    'reg_alpha': [0, 0.1, 0.5],                # L1 regularization term on weights
    'reg_lambda': [1, 1.5, 2]                  # L2 regularization term on weights
}

# ================================
# Step 3: Initialize the XGBoost Classifier
# ================================

# Initialize the classifier with default parameters
xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',   # For binary classification
    use_label_encoder=False,       # Disable label encoder if labels are already in [0, 1]
    eval_metric='logloss',         # Evaluation metric
    random_state=42                # For reproducibility
)

# ================================
# Step 4: Set Up GridSearchCV
# ================================

from sklearn.model_selection import StratifiedKFold

# Use StratifiedKFold for maintaining label distribution in folds
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_clf,
    param_grid=param_grid,
    scoring='roc_auc',         # Use ROC AUC as the scoring metric
    cv=cv,                     # 5-fold cross-validation
    n_jobs=-1,                 # Use all available CPU cores
    verbose=1
)

# ================================
# Step 5: Fit the Model Using Grid Search
# ================================

# Fit the model to the training data
print("Starting grid search...")
grid_search.fit(X_train, y_train)
print("Grid search completed.")

# ================================
# Step 6: Retrieve the Best Parameters and Model
# ================================

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Get the best estimator
best_model = grid_search.best_estimator_

# ================================
# Step 7: Evaluate the Model on the Test Set
# ================================

# Make predictions on the test set
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Evaluate the model
accuracy = accuracy_score(X_test, y_test)
auc = roc_auc_score(y_test, y_pred_proba)
print("\nTest Set Evaluation:")
print("--------------------")
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC AUC Score: {auc:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Calculate confusion matrix components
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Calculate sensitivity (recall) and specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

# ================================
# Step 8: Optional - Analyze Feature Importance
# ================================

# Get feature importance
import matplotlib.pyplot as plt
xgb.plot_importance(best_model, max_num_features=10)
plt.title("Feature Importance")
plt.show()

# ================================
# Step 9: Save the Best Model (Optional)
# ================================

# Save the model for future use
best_model.save_model('best_xgb_model.json')
print("Best model saved as 'best_xgb_model.json'.")

# ================================
# Step 10: Predict on New Data (Optional)
# ================================

# If you have new data to predict
# new_data = pd.read_csv('new_data.csv')
# new_predictions = best_model.predict(new_data)


Starting grid search...
Fitting 5 folds for each of 6561 candidates, totalling 32805 fits


KeyboardInterrupt: 

In [ ]:
model = xgb.XGBClassifier(
    colsample_bytree=0.6,
    gamma=0.2,
    learning_rate=0.1,
    max_depth=7,
    n_estimators=200,
    reg_alpha=0.1,
    reg_lambda=1.5,
    subsample=1.0,
    objective='binary:logistic',   # Set to binary classification
    use_label_encoder=False,       # Disable label encoder if labels are already 0 and 1
    eval_metric='logloss',         # Set evaluation metric for binary classification
    random_state=42                # Ensure reproducibility
)

model_bi.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model_bi.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Evaluate the model_bi
accuracy = accuracy_score(y_pred, y_test)
auc = roc_auc_score(y_test, y_pred_proba)
print("\nTest Set Evaluation:")
print("--------------------")
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC AUC Score: {auc:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Calculate confusion matrix components
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Calculate sensitivity (recall) and specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")



Test Set Evaluation:
--------------------
Accuracy: 0.9094
ROC AUC Score: 0.9216

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       805
           1       0.85      0.84      0.84       332

    accuracy                           0.91      1137
   macro avg       0.89      0.89      0.89      1137
weighted avg       0.91      0.91      0.91      1137

Sensitivity (Recall): 0.8434
Specificity: 0.9366


## adding positive samples from 50k to cryptic

In [29]:
import numpy as np

def select_random_rows_with_labels(array, labels, num_rows, seed=42):
    """
    Randomly selects a specified number of rows from the given array and their corresponding labels.

    Parameters:
    - array (numpy.ndarray): The input array from which to select rows.
    - labels (list or numpy.ndarray): The corresponding items or labels to select.
    - num_rows (int): The number of rows/items to select.
    - seed (int): Random seed for reproducibility.

    Returns:
    - selected_rows (numpy.ndarray): Array containing the randomly selected rows.
    - selected_labels (list or numpy.ndarray): List or array containing the corresponding selected items.
    """
    np.random.seed(seed)  # Set the random seed for reproducibility
    total_rows = array.shape[0]
    
    # Ensure that num_rows does not exceed total_rows
    if num_rows > total_rows:
        raise ValueError("num_rows exceeds the number of rows in the array")
    
    # Ensure that labels and array have the same length
    if len(labels) != total_rows:
        raise ValueError("Labels and array must have the same number of rows")
    
    # Randomly select indices without replacement
    selected_indices = np.random.choice(total_rows, size=num_rows, replace=False)
    
    # Select the rows and labels/items using the indices
    selected_rows = array[selected_indices, :]
    selected_labels = [labels[i] for i in selected_indices]  # For lists
    # If labels is a NumPy array, you can use: selected_labels = labels[selected_indices]
    
    return selected_rows, selected_labels

# Example usage:
# Assuming aa_df_50k_neg is your array with at least 6874 rows
# And mic_series_50k_pos is a list or array of the same length as aa_df_50k_neg

aa_df_50k_neg_selected, mic_series_50k_pos_selected = select_random_rows_with_labels(
    aa_array_50k_neg,
    mic_series_50k_neg,
    num_rows=6874,
    seed=42
)

print("Selected data shape:", aa_df_50k_neg_selected.shape)  # Should output: (6874, number_of_columns)
print("Number of selected items:", len(mic_series_50k_pos_selected))  # Should output: 6874

Selected data shape: (6874, 1710)
Number of selected items: 6874


In [43]:
mic_series_bi_ext = np.concatenate((mic_series_bi.values, mic_series_50k_pos), axis=0)
aa_array_ext = np.concatenate((aa_array, aa_array_50k_pos), axis=0)
# mic_series_bi_ext = np.concatenate((mic_series_bi_ext, mic_series_50k_pos_selected), axis=0)
# aa_array_ext = np.concatenate((aa_array_ext, aa_df_50k_neg_selected), axis=0)

In [ ]:
# with open('snps_cryptic50kpos_emb.npy', 'wb') as f:
#     np.save(f, aa_array_ext)
# with open('drs_crypti50kpos_emb.npy', 'wb') as f:
#     np.save(f, mic_series_bi_ext)


In [44]:
pd.DataFrame(mic_series_bi_ext).value_counts()

1.0    14916
0.0     8042
Name: count, dtype: int64

In [ ]:
# train_data, test_data, train_target, test_target = data_split(aa_array_ext, mic_series_bi_ext)
# X_train, X_test, y_train, y_test = train_data, test_data, train_target, test_target

# train_data, test_data, train_target, test_target = data_split(aa_array, mic_series_bi)
# X_train, X_test, y_train, y_test = train_data, test_data, train_target, test_target


In [57]:
train_data

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split


model = xgb.XGBClassifier(
    # colsample_bytree=0.6,
    # gamma=0.2,        
    # learning_rate=0.1,
    # max_depth=7,
    # n_estimators=200,
    # reg_alpha=0.1,
    # reg_lambda=1.5,
    # subsample=1.0,
    # objective='binary:logistic',   # Set to binary classification
    # use_label_encoder=False,       # Disable label encoder if labels are already 0 and 1
    # eval_metric='logloss',         # Set evaluation metric for binary classification
    # random_state=42                # Ensure reproducibility
)

model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Evaluate the model_bi
accuracy = accuracy_score(y_pred, y_test)
auc = roc_auc_score(y_test, y_pred_proba)
print("\nTest Set Evaluation:")
print("--------------------")
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC AUC Score: {auc:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Calculate confusion matrix components
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(confusion_matrix(y_test, y_pred))
# Calculate sensitivity (recall) and specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")


Test Set Evaluation:
--------------------
Accuracy: 0.9368
ROC AUC Score: 0.9587

Classification Report:
              precision    recall  f1-score   support

         0.0       0.90      0.93      0.91       804
         1.0       0.96      0.94      0.95      1492

    accuracy                           0.94      2296
   macro avg       0.93      0.93      0.93      2296
weighted avg       0.94      0.94      0.94      2296

[[ 746   58]
 [  87 1405]]
Sensitivity (Recall): 0.9417
Specificity: 0.9279


In [ ]:
import pickle
file_name = "xgb_bi_mix.pkl" 

# save
pickle.dump(model, open(file_name, "wb"))

### split the data first and then add the positive samples in training data

In [82]:
train_data, test_data, train_target, test_target = data_split(aa_array, mic_series_bi)
X_train, X_test, y_train, y_test = train_data, test_data, train_target, test_target

X_train = np.concatenate((X_train, aa_array_50k_pos), axis=0)
y_train = np.concatenate((y_train, mic_series_50k_pos), axis=0)

In [84]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

model = xgb.XGBClassifier(
    # colsample_bytree=0.6,
    # gamma=0.2,        
    # learning_rate=0.1,
    # max_depth=7,
    # n_estimators=200,
    # reg_alpha=0.1,
    # reg_lambda=1.5,
    # subsample=1.0,
    # objective='binary:logistic',   # Set to binary classification
    # use_label_encoder=False,       # Disable label encoder if labels are already 0 and 1
    # eval_metric='logloss',         # Set evaluation metric for binary classification
    # random_state=42                # Ensure reproducibility
)

model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Evaluate the model_bi
accuracy = accuracy_score(y_pred, y_test)
auc = roc_auc_score(y_test, y_pred_proba)
print("\nTest Set Evaluation:")
print("--------------------")
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC AUC Score: {auc:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Calculate confusion matrix components
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(confusion_matrix(y_test, y_pred))
# Calculate sensitivity (recall) and specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")


Test Set Evaluation:
--------------------
Accuracy: 0.9068
ROC AUC Score: 0.9172

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.93      0.93       805
           1       0.83      0.86      0.84       332

    accuracy                           0.91      1137
   macro avg       0.88      0.89      0.89      1137
weighted avg       0.91      0.91      0.91      1137

[[746  59]
 [ 47 285]]
Sensitivity (Recall): 0.8584
Specificity: 0.9267


In [85]:
import pickle
file_name = "xgb_bi_mix1.pkl"

# save
pickle.dump(model, open(file_name, "wb"))

In [87]:

with open('snps_crypticTest_emb.npy', 'wb') as f:
    np.save(f, test_data)
with open('snps_crypticTrain_emb.npy', 'wb') as f:
    np.save(f, train_data)
with open('drs_crypticTest_emb.npy', 'wb') as f:
    np.save(f, test_target)
with open('drs_crypticTrain_emb.npy', 'wb') as f:
    np.save(f, train_target)


### trained on balanced 50k data, tested on cryptic

In [60]:
aa_array_50k_pos.shape[0]

11596

In [63]:
aa_df_50k_neg_selected, mic_series_50k_neg_selected = select_random_rows_with_labels(
    aa_array_50k_neg,
    mic_series_50k_neg,
    num_rows=aa_array_50k_pos.shape[0],
    seed=42
)

In [79]:
mic_series_bi_ext = np.concatenate((mic_series_50k_pos, mic_series_50k_neg_selected), axis=0)
aa_array_ext = np.concatenate((aa_array_50k_pos, aa_df_50k_neg_selected), axis=0)
# mic_series_bi_ext = np.concatenate((mic_series_bi_ext, mic_series_50k_pos_selected), axis=0)
# aa_array_ext = np.concatenate((aa_array_ext, aa_df_50k_neg_selected), axis=0)
# train_data, test_data, train_target, test_target = data_split(aa_array_ext, mic_series_bi_ext)
# X_train, X_test, y_train, y_test = train_data, test_data, train_target, test_target



X_train, X_test, y_train, y_test = aa_array_ext, aa_array, mic_series_bi_ext, mic_series_bi

In [80]:

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split


model = xgb.XGBClassifier(
    # colsample_bytree=0.6,
    # gamma=0.2,        
    # learning_rate=0.1,
    # max_depth=7,
    # n_estimators=200,
    # reg_alpha=0.1,
    # reg_lambda=1.5,
    # subsample=1.0,
    # objective='binary:logistic',   # Set to binary classification
    # use_label_encoder=False,       # Disable label encoder if labels are already 0 and 1
    # eval_metric='logloss',         # Set evaluation metric for binary classification
    # random_state=42                # Ensure reproducibility
)

model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Evaluate the model_bi
accuracy = accuracy_score(y_pred, y_test)
auc = roc_auc_score(y_test, y_pred_proba)
print("\nTest Set Evaluation:")
print("--------------------")
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC AUC Score: {auc:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Calculate confusion matrix components
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(confusion_matrix(y_test, y_pred))
# Calculate sensitivity (recall) and specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")


Test Set Evaluation:
--------------------
Accuracy: 0.9092
ROC AUC Score: 0.9025

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      8042
           1       0.84      0.85      0.85      3320

    accuracy                           0.91     11362
   macro avg       0.89      0.89      0.89     11362
weighted avg       0.91      0.91      0.91     11362

[[7506  536]
 [ 496 2824]]
Sensitivity (Recall): 0.8506
Specificity: 0.9333


In [81]:
import pickle
file_name = "xgb_bi_50kbalanced.pkl"    


# save
pickle.dump(model, open(file_name, "wb"))